In [1]:
import sys
import logging
import tomli
import pandas as pd

import pytanis
from pytanis import GSheetClient, PretalxClient
from pytanis.google import Scope, gsheet_rows_for_fmt, worksheet_range
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df
from pytanis.utils import implode


In [2]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

'0.5'

In [3]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)


## Retrieve the talks

In [4]:
pretalx_client = PretalxClient(blocking=True)
talks_count, talks = pretalx_client.submissions(cfg['event_name'], params={"questions": "all", "state": ["confirmed"]})
talks = list(talks)

  0%|          | 0/115 [00:00<?, ?it/s]

In [5]:
def to_pdf(subs):
    rows = []
    for sub in subs:
        if not sub.slot:
            print("Not scheduled?")
            print(sub.speakers, sub.title, sub.code)
            continue
        row = {
            "Submission": sub.code,
            "Speaker": ", ".join([speaker.name for speaker in sub.speakers]),
            "Day": f"{sub.slot.start.day}",
            "Time": f"{sub.slot.start.time()}-{sub.slot.end.time()}",
            "Room": sub.slot.room.en,
            "Duration": sub.duration,
            "Track": sub.track.en if sub.track else None,
            "Title": sub.title,
            "URL": f"https://pretalx.com/pyconde-pydata-berlin-2023/talk/{sub.code}/"
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [6]:
df = to_pdf(talks)

Not scheduled?
[SubmissionSpeaker(code='NMACLQ', name='Tereza Iofciu', biography=None, avatar='https://pretalx.com/media/avatars/IMG_1246_mOIzBWp.jpg', email='terezaif@gmail.com')] Rethinking codes of conduct AWBLKN
Not scheduled?
[SubmissionSpeaker(code='8BJ7S9', name='Lev Konstantinovskiy', biography='Lev Konstantinovskiy is an experienced data science and software engineering team lead. Long time ago he used to maintain a python Natural Language Processing library gensim.', avatar=None, email='lev.konst@gmail.com')] Prompt Engineering 101: Beginner intro to LangChain MSZG7B


In [7]:
rooms = list(df['Room'].unique())
rooms

['Kuppelsaal', 'B05-B06', 'B07-B08', 'A1', 'B09', 'A03-A04', 'A05-A06']

In [8]:
columns_list = ["Day", "Time", "Speaker", "Title", "Duration", "Submission", "URL"]

## Upload to GSheet

In [9]:
gsheet_client = GSheetClient(read_only=False)

In [10]:
for room in rooms:
    gsheet_client.save_df_as_gsheet(df[df['Room']==room][columns_list].sort_values(by=['Day', 'Time']), cfg['schedule_spread_id'], room, create_ws=True)